# Simulation

CoHDL is currently not able to simulate synthesizable contexts. Instead [cohdl_sim](https://github.com/alexander-forster/cohdl_sim) can be used. This Python module uses [cocotb](https://www.cocotb.org/) to define testbenches for the generated output of CoHDL.

The advantage of this approach is, that the designs are tested using established tools.

The following simple example defines tests for an entity that adds two numbers. For more details checkout the cohdl_sim repo.

In [1]:
from cohdl import Entity, Port, Unsigned
from cohdl import std

class MyDesign(Entity):
    inp_a = Port.input(Unsigned[8])
    inp_b = Port.input(Unsigned[8])

    out_sum = Port.output(Unsigned[8])

    def architecture(self):
        @std.concurrent
        def interleave_logic():
            self.out_sum <<= self.inp_a + self.inp_b

cohdl_sim provides a single class `Simulator`. It wraps a CoHDL entity, translates it to VHDL and starts a cocotb test session with that code. Additionally it abstracts cocotb functions so testbenches look like synthesizable CoHDL code.

In [2]:
from cohdl_sim.ghdl_sim import Simulator

# ghdl_sim is a special Simulator backend that is only available under Linux
# and requires some additional setup. The main advantage over the default simulator
# is, that it can run in jupyter notebooks.

# To use the default simulator based on cocotb, uncomment the following import.
# Check the cohdl_sim repo for more information.
# from cohdl_sim import Simulator

sim = Simulator(MyDesign)

@sim.test
async def testbench_1(entity: MyDesign):
    entity.inp_a <<= 1
    entity.inp_b <<= 2

    print(f"initial: {entity.out_sum}")

    # run simulator to update output
    await sim.delta_step()

    assert entity.out_sum == 3

    print(f"result: {entity.out_sum}")


initial: 0u
result: 3u


loading VPI module '/home/alexander/dev/cohdl/cohdl_documentation/.venv/lib/python3.11/site-packages/cohdl_sim_ghdl_interface.cpython-311-x86_64-linux-gnu.so'
VPI module loaded!


In [3]:
@sim.test
async def testbench_2(entity: MyDesign):
    for a in range(5):
        for b in range(5):
            entity.inp_a <<= a
            entity.inp_b <<= b

            await sim.delta_step()

            assert entity.out_sum == a+b
            print(f"{entity.out_sum} == {a} + {b}")

0u == 0 + 0
1u == 0 + 1
2u == 0 + 2
3u == 0 + 3
4u == 0 + 4
1u == 1 + 0
2u == 1 + 1
3u == 1 + 2
4u == 1 + 3
5u == 1 + 4
2u == 2 + 0
3u == 2 + 1
4u == 2 + 2
5u == 2 + 3
6u == 2 + 4
3u == 3 + 0
4u == 3 + 1
5u == 3 + 2
6u == 3 + 3
7u == 3 + 4
4u == 4 + 0
5u == 4 + 1
6u == 4 + 2
7u == 4 + 3
8u == 4 + 4


loading VPI module '/home/alexander/dev/cohdl/cohdl_documentation/.venv/lib/python3.11/site-packages/cohdl_sim_ghdl_interface.cpython-311-x86_64-linux-gnu.so'
VPI module loaded!
